In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install matplotlib


Looking in indexes: https://download.pytorch.org/whl/cu121


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
DATASET_PATH = "/content/drive/MyDrive/MyDrive dataset"

In [ ]:
!ls "/content/drive/MyDrive"

 archive	   'Extra sheet chemical reaction pdf.gdoc'
 archive.zip	   'Getting started.pdf'
'Brava AI.pptx'    'MyDrive dataset'
 Classroom	   'Omar ALSAYEGH CV (1).docx'
'Colab Notebooks'  'Unit 1-chapter 3.gslides'
 dataset_split	   'Untitled form.gform'


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import time

In [6]:
import os
import shutil
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 4️⃣ Paths
SOURCE_DIR = "/content/drive/MyDrive/MyDrive dataset/dataset"        # original dataset
TARGET_DIR = "/content/drive/MyDrive/dataset_split"          # where split will be stored

train_dir = os.path.join(TARGET_DIR, "train")
val_dir   = os.path.join(TARGET_DIR, "val")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# 5️⃣ Split dataset into train/val on disk
train_ratio = 0.8

classes = [cls for cls in os.listdir(SOURCE_DIR) if os.path.isdir(os.path.join(SOURCE_DIR, cls))]

for cls in classes:
    cls_path = os.path.join(SOURCE_DIR, cls)

    images = [f for f in os.listdir(cls_path) if f.lower().endswith(("jpg", "jpeg", "png"))]
    if not images:
        print(f"No images found in {cls_path}, skipping...")
        continue

    train_imgs, val_imgs = train_test_split(images, test_size=1-train_ratio, random_state=42)

    # Create class folders in train/val
    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)

    # Copy files
    for img in train_imgs:
        shutil.copy(os.path.join(cls_path, img), os.path.join(train_dir, cls, img))
    for img in val_imgs:
        shutil.copy(os.path.join(cls_path, img), os.path.join(val_dir, cls, img))

print("Dataset split completed successfully!")

Dataset split completed successfully!


In [7]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.6, 1.0)),
    transforms.RandomRotation(25),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.02),
    transforms.RandomPerspective(distortion_scale=0.4, p=0.7),
    transforms.RandomHorizontalFlip(),
    transforms.GaussianBlur(3),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.4),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [8]:
train_ds = datasets.ImageFolder(train_dir, transform=train_transforms)
val_ds   = datasets.ImageFolder(val_dir, transform=val_transforms)

In [9]:
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=16, shuffle=False)

In [10]:
NUM_CLASSES = len(train_ds.classes)
print("Classes:", train_ds.classes)
print("Number of training images:", len(train_ds))
print("Number of validation images:", len(val_ds))

Classes: ["Assassin's_Creed_Origins_cover_PS4", 'CS2_Cover', 'Dota_2_Cover', 'Fortnite_Deluxe_Edition', 'GTA_V_poster', 'League_of_Legends_Cover', 'Overwatch_2_Cover', 'Valorant_Cover']
Number of training images: 75
Number of validation images: 29


In [11]:
model = models.efficientnet_b0(weights='IMAGENET1K_V1')

for param in model.features.parameters():
    param.requires_grad = False  # Freeze base

model.classifier[1] = nn.Linear(model.classifier[1].in_features, NUM_CLASSES)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 158MB/s]


In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.0008)

In [14]:
EPOCHS = 80

train_losses = []
val_losses = []

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    train_loss = running_loss / len(train_loader)
    train_losses.append(train_loss)

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            pred = outputs.argmax(dim=1)
            correct += (pred == labels).sum().item()

    val_loss /= len(val_loader)
    val_losses.append(val_loss)

    accuracy = correct / len(val_ds)

    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Acc: {accuracy:.4f}")

Epoch 1/80 | Train Loss: 1.9566 | Val Loss: 1.8111 | Acc: 0.4138
Epoch 2/80 | Train Loss: 1.8436 | Val Loss: 1.7043 | Acc: 0.6207
Epoch 3/80 | Train Loss: 1.7892 | Val Loss: 1.6143 | Acc: 0.6897
Epoch 4/80 | Train Loss: 1.6417 | Val Loss: 1.5517 | Acc: 0.6552
Epoch 5/80 | Train Loss: 1.5307 | Val Loss: 1.4734 | Acc: 0.7586
Epoch 6/80 | Train Loss: 1.5459 | Val Loss: 1.3987 | Acc: 0.7931
Epoch 7/80 | Train Loss: 1.4322 | Val Loss: 1.3449 | Acc: 0.8966
Epoch 8/80 | Train Loss: 1.4292 | Val Loss: 1.2695 | Acc: 0.8621
Epoch 9/80 | Train Loss: 1.2901 | Val Loss: 1.2156 | Acc: 0.8621
Epoch 10/80 | Train Loss: 1.3031 | Val Loss: 1.1508 | Acc: 0.8966
Epoch 11/80 | Train Loss: 1.1918 | Val Loss: 1.1150 | Acc: 0.8966
Epoch 12/80 | Train Loss: 1.2086 | Val Loss: 1.0892 | Acc: 0.8621
Epoch 13/80 | Train Loss: 1.0684 | Val Loss: 1.0581 | Acc: 0.8621
Epoch 14/80 | Train Loss: 1.0377 | Val Loss: 1.0494 | Acc: 0.8966
Epoch 15/80 | Train Loss: 1.0078 | Val Loss: 1.0164 | Acc: 0.8621
Epoch 16/80 | Train

In [15]:
MODEL_PATH = "/content/drive/MyDrive/game_model.pth"
torch.save(model.state_dict(), MODEL_PATH)

print("Model saved to:", MODEL_PATH)

Model saved to: /content/drive/MyDrive/game_model.pth


In [ ]:
from PIL import Image
import torch.nn.functional as F
from google.colab import files

# 1️⃣ Upload an image from your computer
uploaded = files.upload()  # This will open a file picker
filename = list(uploaded.keys())[0]  # Get the uploaded file name

# 2️⃣ Define the predict function
def predict(image_path):
    img = Image.open(image_path).convert("RGB")
    img = val_transforms(img).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        output = model(img)
        probs = F.softmax(output, dim=1)
        label_index = probs.argmax().item()
        label = train_ds.classes[label_index]
        return label, probs[0][label_index].item()

# 3️⃣ Use the uploaded image for prediction
label, confidence = predict(filename)
print("Prediction:", label, "| Confidence:", confidence)
